### IEX Code -------------------------------------------------------
1. Define the stock attributes
2. Define functions: filter attributes, fetch stock data, log events
3. Read company list from the redis datastore
4. Setup schedule
5. Fetch data at scheduled event.

In [1]:
#Imports
import redis
import urllib.request
import json
import numpy as np
import pandas as pd
import schedule
import time
import boto3
from datetime import date, datetime
import traceback
import os

kinesis = boto3.client('kinesis', region_name='us-east-1')

#Mongo
from pymongo import MongoClient


#Connect to Redis-DataStore
REDIS = redis.Redis(host='data_store')

In [2]:
#Setup Mongo and create the database and collection
client = MongoClient('db-data')
db = client['stock_tweets']
coll_reference = db.iex

In [3]:
########### Set up Feature Flag #####################

   
def get_feature_flag(feature):
    all_flags = pd.read_msgpack(REDIS.get("feature_flags"))
    
    try:
        return all_flags.get_value(feature, 'State')
        
    except:
        return 'Flag Not Found, not a valid feature flag'

In [4]:
############ Define Functions #################################
attributes = ['latestUpdate',
             'companyName',
             'latestPrice',
              'latestVolume',
              'marketCap',
              'open',
              'previousClose',
              'sector',
              'high',
              'low',
              'ytdChange',
              'peRatio',
              'week52High',
              'week52Low'
               ]

#Select interesting stock attributes.
def filter_stock_attributes(data):
    output = {}
    #Choose filter attributes
    for element in attributes:
        output[element]=data[element]
    
    #Convert time
    ctime = output['latestUpdate']/1000
    new_time = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(ctime))
    output['latestUpdate']= new_time
    
    return output


#Create function to fetch the stock data. This is to prepare for the schedule.

def fetch_stock_data(stocks=[]):
    try:
        #Only pull stock info if the DataFlag is set.
        if int(REDIS.get('Data_On')) == 1:

            for ticker in stocks:
                url = "https://api.iextrading.com/1.0/stock/{}/quote".format(ticker)
                response = urllib.request.urlopen(url)
                str_response = response.read().decode('utf-8')
                obj = json.loads(str_response)
                filtered = filter_stock_attributes(obj)
                filtered['Ticker'] = ticker
                #Add counter to count stocks. 
                REDIS.incr('IEX_Stock_Count')
                #<------ Insert into MongoDb ----------->
                if int(get_feature_flag('database_stream_write'))==1:
                        coll_reference.insert_one(filtered)
                
                #<----- Insert to Kinesis Stream ------->
                if int(get_feature_flag('kinesis_stream_write'))==1:
                    response = kinesis.put_record(StreamName="IEX_Stream", Data=json.dumps(filtered), PartitionKey="partitionkey")
                print('---------------------------------')
                #print(response)
                print(filtered)
    except:
        print(traceback.format_exc())
        #Send Error event
        send_event('IEX', 'Error', 'Error occured, check stdout')

#Setup Log

global past_stock_count
past_stock_count = 0




########### Redis Subscription setup ########################
queue = REDIS.pubsub()
#Subscribe to the queues one for the events and one for the log
queue.subscribe('event_queue')
queue.subscribe('log_queue')


#Serialize datetime.
def json_serial(obj):
    """JSON serializer for objects not serializable by default json code"""

    if isinstance(obj, (datetime, date)):
        return obj.isoformat()
    raise TypeError ("Type %s not serializable" % type(obj))
    


#Code to log to the event queue
def send_event(source, kind, message):
    event_time = datetime.now()
    event_time = json_serial(event_time)
    event = {
            "event_time": event_time,
            "source": source,
            "kind" : kind,
            "message" : message
            }
    payload = json.dumps(event)
    REDIS.publish('event_queue', payload)

def send_log(source, current_count, count_diff):
    log_time = datetime.now()
    log_time = json_serial(log_time)
    log = {
            "log_time": log_time,
            "source": source,
            "current_count" : current_count,
            "count_diff" : count_diff
            }
    payload = json.dumps(log)
    REDIS.publish('log_queue', payload)


#Send the log data to the Redis channel.
def log():
    #Need to log: Time, Source, Current Count, Count Diff
    #now = datetime.datetime.now()
    current_stock_count = int(REDIS.get('IEX_Stock_Count'))    
     
    global past_stock_count
    stock_count_diff = current_stock_count - past_stock_count
    past_stock_count = current_stock_count
    
    #Send the log event
    send_log(source='IEX',current_count = current_stock_count, count_diff=stock_count_diff)
    
    print('Logged Data')

In [ ]:
#Read Data-Store
companies = json.loads(REDIS.get('companies').decode())
stock_tickers = list(companies.keys())
REDIS.set('IEX_Stock_Count', 0)

#Setup Schedule
schedule.clear()
schedule.every(30).seconds.do(fetch_stock_data, stocks=stock_tickers)
schedule.every(35).seconds.do(log)

#Send Start event
send_event('IEX', 'Activity', 'Data Source Started')

#Execute
while True:
    schedule.run_pending()
    time.sleep(1)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


---------------------------------
{'latestUpdate': '2018-03-08 15:56:47', 'companyName': 'Apple Inc.', 'latestPrice': 175.53, 'latestVolume': 7591724, 'marketCap': 890641501890, 'open': 175.41, 'previousClose': 175.03, 'sector': 'Technology', 'high': 176.089, 'low': 175.07, 'ytdChange': 0.016080343666550624, 'peRatio': 19.08, 'week52High': 180.615, 'week52Low': 137.05, 'Ticker': 'AAPL', '_id': ObjectId('5aa15d43cd1dc9000de88d7c')}
---------------------------------
{'latestUpdate': '2018-03-08 15:56:47', 'companyName': 'Facebook Inc.', 'latestPrice': 182.61, 'latestVolume': 5512512, 'marketCap': 530482188418, 'open': 183.59, 'previousClose': 183.71, 'sector': 'Technology', 'high': 184.4, 'low': 182.1, 'ytdChange': 0.012622643589461033, 'peRatio': 29.69, 'week52High': 195.32, 'week52Low': 137.0516, 'Ticker': 'FB', '_id': ObjectId('5aa15d43cd1dc9000de88d7d')}
---------------------------------
{'latestUpdate': '2018-03-08 15:51:00', 'companyName': 'Alphabet Inc.', 'latestPrice': 1118.42, '

---------------------------------
{'latestUpdate': '2018-03-08 15:56:38', 'companyName': 'Citigroup Inc.', 'latestPrice': 73.475, 'latestVolume': 3305892, 'marketCap': 188835580834, 'open': 74.18, 'previousClose': 73.92, 'sector': 'Financial Services', 'high': 74.2, 'low': 73.13, 'ytdChange': -0.005917159763313579, 'peRatio': 13.79, 'week52High': 80.7, 'week52Low': 56.55, 'Ticker': 'C', '_id': ObjectId('5aa15d47cd1dc9000de88d8f')}
---------------------------------
{'latestUpdate': '2018-03-08 15:56:35', 'companyName': 'United Technologies Corporation', 'latestPrice': 130.43, 'latestVolume': 521254, 'marketCap': 104315083017, 'open': 131.56, 'previousClose': 130.93, 'sector': 'Industrials', 'high': 132.09, 'low': 130.14, 'ytdChange': 0.021613607990012565, 'peRatio': 19.58, 'week52High': 139.24, 'week52Low': 107.05, 'Ticker': 'UTX', '_id': ObjectId('5aa15d47cd1dc9000de88d90')}
---------------------------------
{'latestUpdate': '2018-03-08 15:56:43', 'companyName': 'Comcast Corporation Cl

---------------------------------
{'latestUpdate': '2018-03-08 15:57:43', 'companyName': 'Bank of America Corporation', 'latestPrice': 31.92, 'latestVolume': 15114099, 'marketCap': 326978549560, 'open': 32.19, 'previousClose': 32.18, 'sector': 'Financial Services', 'high': 32.26, 'low': 31.81, 'ytdChange': 0.07625418060200673, 'peRatio': 17.54, 'week52High': 32.85, 'week52Low': 22.07, 'Ticker': 'BAC', '_id': ObjectId('5aa15d88cd1dc9000de88d9e')}
---------------------------------
{'latestUpdate': '2018-03-08 15:57:50', 'companyName': 'The Boeing Company', 'latestPrice': 347.57, 'latestVolume': 1229133, 'marketCap': 204541578089, 'open': 349.51, 'previousClose': 347.04, 'sector': 'Industrials', 'high': 352.5, 'low': 346.87, 'ytdChange': 0.16911467457216026, 'peRatio': 28.77, 'week52High': 371.6, 'week52Low': 173.75, 'Ticker': 'BA', '_id': ObjectId('5aa15d88cd1dc9000de88d9f')}
---------------------------------
{'latestUpdate': '2018-03-08 15:57:47', 'companyName': 'Netflix Inc.', 'latestP

---------------------------------
{'latestUpdate': '2018-02-26 00:00:00', 'companyName': 'The Priceline Group Inc.', 'latestPrice': 1905.64, 'latestVolume': 566730, 'marketCap': 92937197639, 'open': 1905.95, 'previousClose': 1905.64, 'sector': 'Consumer Cyclical', 'high': 1923.22, 'low': 1897.66, 'ytdChange': 0.06773497688751931, 'peRatio': 25.6, 'week52High': 2067.99, 'week52Low': 1612.41, 'Ticker': 'PCLN', '_id': ObjectId('5aa15d8fcd1dc9000de88db1')}
---------------------------------
{'latestUpdate': '2018-03-08 15:57:41', 'companyName': 'Alibaba Group Holding Limited', 'latestPrice': 187.76, 'latestVolume': 5752614, 'marketCap': 481761071826, 'open': 189.32, 'previousClose': 189.05, 'sector': 'Consumer Cyclical', 'high': 190.23, 'low': 187.41, 'ytdChange': 0.02940375714674656, 'peRatio': 71.39, 'week52High': 206.2, 'week52Low': 102.54, 'Ticker': 'BABA', '_id': ObjectId('5aa15d8fcd1dc9000de88db2')}
---------------------------------
{'latestUpdate': '2018-03-08 15:54:35', 'companyName

---------------------------------
{'latestUpdate': '2018-03-08 15:58:33', 'companyName': 'Intel Corporation', 'latestPrice': 51.04, 'latestVolume': 9118356, 'marketCap': 238254720000, 'open': 51.79, 'previousClose': 51.32, 'sector': 'Technology', 'high': 52.08, 'low': 51.03, 'ytdChange': 0.09541088580576304, 'peRatio': 14.71, 'week52High': 51.485, 'week52Low': 33.23, 'Ticker': 'INTC', '_id': ObjectId('5aa15db5cd1dc9000de88dc5')}
---------------------------------
{'latestUpdate': '2018-03-08 15:58:00', 'companyName': 'Home Depot Inc. (The)', 'latestPrice': 178.6, 'latestVolume': 834522, 'marketCap': 208559903353, 'open': 179.44, 'previousClose': 178.58, 'sector': 'Consumer Cyclical', 'high': 179.98, 'low': 178.32, 'ytdChange': -0.050257937563154755, 'peRatio': 27.69, 'week52High': 207.605, 'week52Low': 144.25, 'Ticker': 'HD', '_id': ObjectId('5aa15db6cd1dc9000de88dc6')}
---------------------------------
{'latestUpdate': '2018-03-08 15:58:35', 'companyName': 'Citigroup Inc.', 'latestPric

---------------------------------
{'latestUpdate': '2018-03-08 15:59:13', 'companyName': 'JP Morgan Chase & Co.', 'latestPrice': 113.695, 'latestVolume': 3482572, 'marketCap': 390196520634, 'open': 114.93, 'previousClose': 114.73, 'sector': 'Financial Services', 'high': 115.23, 'low': 113.21, 'ytdChange': 0.06280685502547477, 'peRatio': 16.27, 'week52High': 119.33, 'week52Low': 81.635, 'Ticker': 'JPM', '_id': ObjectId('5aa15dd7cd1dc9000de88dd9')}
---------------------------------
{'latestUpdate': '2018-03-08 15:59:00', 'companyName': 'Tesla Inc.', 'latestPrice': 329.62, 'latestVolume': 1243540, 'marketCap': 55679390952, 'open': 332.94, 'previousClose': 332.3, 'sector': 'Consumer Cyclical', 'high': 333.3, 'low': 328.1, 'ytdChange': 0.036720431784856455, 'peRatio': -28.81, 'week52High': 389.61, 'week52Low': 242.781, 'Ticker': 'TSLA', '_id': ObjectId('5aa15dd7cd1dc9000de88dda')}
---------------------------------
{'latestUpdate': '2018-03-08 15:58:56', 'companyName': 'Cisco Systems Inc.', 

Logged Data
